<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install all the dependencies. Place any new installables into requirements.txt file.
!pip install -r /content/drive/MyDrive/ColabNotebooks/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch

In [4]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Select an audio file and read it:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = ds[0]["audio"]

# Load the Whisper model with SDPA attention
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en", attn_implementation="sdpa")

# Enable static cache and compile the forward pass
model.generation_config.cache_implementation = "static"
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

# Use the model and processor to transcribe the audio:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features

# Compile the forward pass
for _ in range(2):
    model.generate(input_features)

# Generate token ids using compiled graph (fast!)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcription[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'

In [5]:
from transformers import AutoProcessor, Wav2Vec2BertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertForCTC.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

librispeech_asr_demo.py:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

17.04

In [7]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

transcription = whisper("/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav", return_timestamps=True)

print(transcription["text"])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Good afternoon, everyone, and welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from our scalable business model. Our EBITDA has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially, thanks to the expansion of our high-yield savings product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in collateralized debt obligations and residential mortgage-backed securities. We've also invested $25 million in AAA-rated corporate bonds, enhancing our risk-adjusted returns. As for our balance sheet, total assets reached $1.5 

In [9]:
# import torch
# from transformers import pipeline
# import gradio as gr

# def Speech2TextWhisperLarge(audioFile) :
#     whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")
#     transcription = whisper("/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav", return_timestamps=True)
#     #print(transcription["text"])
#     return transcription



# demo = gr.Interface(
#         fn=Speech2TextWhisperLarge,
#         title="Speech to Text",
#         inputs=["file",
#               #  gr.Radio(choices=["Yes", "No"], label="Task", value="Generate"),
#                 ],
#        outputs=["text"])

# demo.launch(debug=True, show_error=True)

# # demo = gr.Interface(
# #         fn=regression_transformer,
# #         title="Regression Transformer",
# #         inputs=[
# #             gr.Dropdown(rt_names, label="Algorithm version", value="Molecules: Qed"),
# #             gr.Radio(choices=["Predict", "Generate"], label="Task", value="Generate"),
# #             gr.Textbox(
# #                 label="Input", placeholder="CC(C#C)N(C)C(=O)NC1=CC=C(Cl)C=C1", lines=1
# #             ),
# #             gr.Slider(
# #                 minimum=1, maximum=50, value=10, label="Number of samples", step=1
# #             ),
# #             gr.Radio(choices=["Sample", "Greedy"], label="Search", value="Sample"),
# #             gr.Slider(minimum=0.5, maximum=2, value=1, label="Decoding temperature"),
# #             gr.Slider(minimum=5, maximum=100, value=30, label="Tolerance", step=1),
# #             gr.Radio(choices=[True, False], label="Sampling Wrapper", value=True),
# #             gr.Slider(minimum=0, maximum=1, value=0.5, label="Fraction to mask"),
# #             gr.Textbox(label="Property goal", placeholder="<qed>:0.75", lines=1),
# #             gr.Textbox(label="Tokens to mask", placeholder="N, C", lines=1),
# #             gr.Textbox(
# #                 label="Substructures to mask", placeholder="C(=O), C#C", lines=1
# #             ),
# #             gr.Textbox(
# #                 label="Substructures to keep", placeholder="C1=CC=C(Cl)C=C1", lines=1
# #             ),
# #         ],
# #         outputs=gr.HTML(label="Output"),
# #         article=article,
# #         description=description,
# #         examples=examples.values.tolist(),
# #     )
# #     demo.launch(debug=True, show_error=True)

In [10]:
!pip install deepspeech

ERROR: Could not find a version that satisfies the requirement deepspeech (from versions: none)
ERROR: No matching distribution found for deepspeech


In [13]:
import gradio as gr
import torch
from transformers import pipeline
import time  # <-- Added

# Function to transcribe with Whisper
def transcribe_audio_with_whisper(file):
    whisper = pipeline(
        "automatic-speech-recognition",
        "openai/whisper-large-v3",
        torch_dtype=torch.float16,
        device="cuda:0"
    )
    transcription = whisper(file, return_timestamps=True)
    return transcription["text"]

# Function to transcribe with Wav2vec2
def transcribe_audio_with_wav2vec2(file):
    # Load the Wav2Vec2 model
    asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-960h-lv60-self")

    # Run transcription
    transcription = asr(file)

    # Return just the transcribed text
    return transcription["text"]

# Function to transcribe with whisper_small
def transcribe_audio_with_whisper_small(file):
    # Load the Whisper small model
    whisper_small = pipeline("automatic-speech-recognition", model="openai/whisper-small")

    # Transcribe the audio
    transcription = whisper_small(file)

    # Return only the text
    return transcription["text"]

# Function to transcribe with deepspeech

# import deepspeech
# import wave
# import numpy as np

# def transcribe_audio_with_deepspeech(file, model_file, scorer_file):
#     # Load DeepSpeech model
#     model = deepspeech.Model(model_file)
#     model.enableExternalScorer(scorer_file)

#     # Read audio file
#     with wave.open(audio_path, 'rb') as wf:
#         frames = wf.getnframes()
#         buffer = wf.readframes(frames)
#         audio = np.frombuffer(buffer, dtype=np.int16)

#     # Perform speech-to-text
#     text = model.stt(audio)
#     return text

# model_path = 'deepspeech-0.9.3-models.pbmm'
# scorer_path = 'deepspeech-0.9.3-models.scorer'
# #audio_path = '/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav'

# Function to transcribe with Speech recognition

!pip install SpeechRecognition

import speech_recognition as sr

def transcribe_audio_with_google_sr(file):
    recognizer = sr.Recognizer()

    # Load and process audio file
    with sr.AudioFile(file) as source:
        audio = recognizer.record(source)

    # Transcribe using Google Web Speech API (requires internet)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

# # Other processing functions
# def process_file_option_1(file):
#     return f"Option 1: File '{file.name}' uploaded and processed."

# def process_file_option_2(file):
#     return f"Option 2: File '{file.name}' processed in a different way."

# def process_file_option_3(file):
#     return f"Option 3: Custom logic applied to file '{file.name}'."

# Dispatcher with timing
def process_based_on_selection(file, choice):
    start_time = time.time()

    if choice == "Whisper-large-v3":
        result = transcribe_audio_with_whisper(file)
    elif choice == "Facebook - Wav2vec2":
        result = transcribe_audio_with_wav2vec2(file)
    elif choice == "Whisper-Small":
        result = transcribe_audio_with_whisper_small(file)
    # elif choice == "Mozzilla - Deepspeech":
    #     result = transcribe_audio_with_deepspeech(file, model_path, scorer_path)
    elif choice == "Google - Speech Recognition":
        result = transcribe_audio_with_google_sr(file)
    else:
        result = "Invalid selection."

    end_time = time.time()
    duration = end_time - start_time

    return f"{result}\n\n⏱️ Processing time: {duration:.2f} seconds"

# Options
options = ["Whisper-large-v3", "Facebook - Wav2vec2", "Whisper-Small", "Mozzilla - Deepspeech", "Google - Speech Recognition"]

# Gradio interface with split layout
with gr.Blocks() as demo:
    gr.Markdown("### File Processor App")

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Upload your file")
            dropdown = gr.Dropdown(choices=options, label="Choose processing option")
            submit_btn = gr.Button("Submit")

        with gr.Column(scale=1):
            output = gr.Textbox(label="Result", lines=10)

    submit_btn.click(fn=process_based_on_selection, inputs=[file_input, dropdown], outputs=output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d41e95e273d0810c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
!pip install gradio
import gradio as gr
import torch
from transformers import pipeline
import time  # <-- Added

# Function to transcribe with whisper-large-v3
def transcribe_audio_with_whisper(file):
    whisper = pipeline(
        "automatic-speech-recognition",
        "openai/whisper-large-v3",
        torch_dtype=torch.float16,
        device="cuda:0"
    )
    transcription = whisper(file, return_timestamps=True)
    return transcription["text"]

from transformers import pipeline

# # Function to transcribe with Wav2vec2
# def transcribe_audio_with_wav2vec2(audio_path):
#     # Load the Wav2Vec2 model
#     asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-960h-lv60-self")

#     # Run transcription
#     transcription = asr(audio_path)

#     # Return just the transcribed text
#     return transcription["text"]

# # Function to transcribe with whisper_small
# from transformers import pipeline

# def transcribe_audio_with_whisper_small(audio_path):
#     # Load the Whisper small model
#     whisper_small = pipeline("automatic-speech-recognition", model="openai/whisper-small")

#     # Transcribe the audio
#     transcription = whisper_small(audio_path)

#     # Return only the text
#     return transcription["text"]



# Other processing functions
def process_file_option_1(file):
    return f"Option 1: File '{file.name}' uploaded and processed."

def process_file_option_2(file):
    return f"Option 2: File '{file.name}' processed in a different way."

def process_file_option_3(file):
    return f"Option 3: Custom logic applied to file '{file.name}'."

# Dispatcher with timing
def process_based_on_selection(file, choice):
    start_time = time.time()

    if choice == "Whisper-large-v3":
        result = transcribe_audio_with_whisper(file)

    elif choice == "Process with Option 2":
        result = process_file_option_2(file)

    elif choice == "Process with Option 3":
        result = process_file_option_3(file)
    else:
        result = "Invalid selection."

    end_time = time.time()
    duration = end_time - start_time

    return f"{result}\n\n⏱️ Processing time: {duration:.2f} seconds"

# Options
options = ["Process with Option 1", "Process with Option 2", "Process with Option 3"]

# Gradio interface with split layout
with gr.Blocks() as demo:
    gr.Markdown("### Minutes of Meeting")

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Upload your file")
            dropdown = gr.Dropdown(choices=options, label="Select the model to be used")
            submit_btn = gr.Button("Submit")

        with gr.Column(scale=1):
            output = gr.Textbox(label="Transcripted text", lines=10)

    submit_btn.click(fn=process_based_on_selection, inputs=[file_input, dropdown], outputs=output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f1766d7c625adefc27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
print(transcription["text"])

In [ ]:
pip install gradio

In [ ]:
from pydub import AudioSegment
import torch
from transformers import pipeline

# Load Whisper model
whisper = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-large-v3",
    torch_dtype=torch.float16,
    device="cuda:0"
)

# Load the audio file
audio = AudioSegment.from_wav("/content/drive/MyDrive/Colab Notebooks/EarningsCall.wav")

# Define chunk length (e.g., 25 seconds)
chunk_length_ms = 25000  # 25 seconds
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# Save & Transcribe each chunk
transcriptions = []
for idx, chunk in enumerate(chunks):
    chunk.export(f"chunk_{idx}.wav", format="wav")
    result = whisper(f"chunk_{idx}.wav")  # Transcribe
    transcriptions.append(result["text"])

# Combine the transcriptions
full_transcription = " ".join(transcriptions)
print(full_transcription)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

print(tokenizer.backend_tokenizer.normalizer.normalize_str(full_transcription))

In [ ]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(full_transcription)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(full_transcription)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print(summarizer(full_transcription, max_length=130, min_length=30, do_sample=False))

In [ ]:
summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)
print(summary)

In [ ]:

# # Function to extract action items
# import re

# def extract_action_items(full_transcription):
#     action_keywords = ["must", "should", "need to", "required", "ensure", "action item", "task", "responsible for", "deadline"]
#     sentences = re.split(r'(?<=[.!?])\s+', full_transcription)
#     action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]
#     return action_items

# # Extract action items from the transcribed text
# action_items = extract_action_items(transcription[0])

# # Display extracted action items
# print("Extracted Action Items:")
# for idx, item in enumerate(action_items, start=1):
#     print(f"{idx}. {item}")


In [ ]:
# # Function to extract action items
# import re
# print(full_transcription)
# def extract_action_items(full_transcription):
#     action_keywords = ["must", "should", "need to", "Investing", "Expanding", "Diversifying", "Enhancing"]
#     sentences = re.split(r'(?<=[.!?])\s+', full_transcription)
#     action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]
#     return action_items

# # Extract action items from the transcribed text
# # Access the 'text' key of the transcription dictionary
# action_items = extract_action_items(full_transcription)

# # Display extracted action items
# print("Extracted Action Items:")
# for idx, item in enumerate(action_items, start=1):
#     print(f"{idx}. {item}")

In [ ]:
import re

def extract_action_items(full_transcription):
    """
    Extracts action items from the given transcribed text by identifying sentences with action-oriented keywords.
    """
    # Keywords that indicate action items
    action_keywords = ["must", "should", "need to", "Investing", "Expanding", "Diversifying", "Enhancing", "revenue", "risk", "loss"]

    # Split text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', full_transcription)

    # Extract sentences that contain action-related words
    action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]

    return action_items

# Example transcribed text (Replace this with your actual transcription)
# full_transcription1 = """
# The team needs to complete the project report by next Monday.
# John is responsible for finalizing the budget.
# Ensure that all client feedback is incorporated before submission.
# We had a general discussion about the upcoming product launch.
# Sarah should follow up with the vendor for the new software license.
# """

# Extract action items
action_items = extract_action_items(full_transcription)

# Print extracted action items
print("Extracted Action Items:")
for idx, item in enumerate(action_items, start=1):
    print(f"{idx}. {item}")


In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
load_recall = evaluate.load("recall")
load_f1 = evaluate.load("f1")

In [ ]:
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Download the 'punkt' and 'punkt_tab' resources
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
#Evaluating metrics with full_transcription

import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

nltk.download('punkt')

# Sample reference summary (manually written or an official meeting summary)
#reference_summary_sample = ["The team must complete the project by next week. The budget needs approval."]

# Transcribed text (output from Whisper model)
#full_transcription_sample = "The project completion deadline is next week. The team is responsible. Budget needs to be checked."

# Tokenize reference and generated text
reference_tokens = [nltk.word_tokenize(summary[0]['summary_text'])]
transcribed_tokens = nltk.word_tokenize(full_transcription)

# Compute BLEU Score
bleu_score = sentence_bleu(reference_tokens, transcribed_tokens, smoothing_function=SmoothingFunction().method1)

# Compute ROUGE Score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(summary[0]['summary_text'], full_transcription)

# Convert text to binary vectors for metric calculations
ref_words = set(reference_tokens[0])
trans_words = set(transcribed_tokens)
all_words = list(ref_words.union(trans_words))

y_true = [1 if word in ref_words else 0 for word in all_words]
y_pred = [1 if word in trans_words else 0 for word in all_words]

# Compute F1 Score
f1 = f1_score(y_true, y_pred)

# Compute Recall
recall = recall_score(y_true, y_pred)

# Compute Precision
precision = precision_score(y_true, y_pred)

# Compute Accuracy
accuracy = accuracy_score(y_true, y_pred)

print("\n\n")
# Print Evaluation Metrics
print(f"\nBLEU Score: {bleu_score:.4f}")
print(f"\nPrecision Score: {precision:.4f}")
print(f"\nAccuracy Score: {accuracy:.4f}")
print(f"\nF1 Score: {f1:.4f}")
print(f"\nROUGE Scores: {rouge_scores}")
print(f"\nRecall Score: {recall:.4f}")
